## 時系列を考慮した適切なモデル評価（スタンドアロン版）

データリークを防ぎ、実際の運用を想定した評価を行います。

In [31]:
# 必要なライブラリのインポート
try:
    import lightgbm as lgb
    import pandas as pd
    import numpy as np
    from sklearn.metrics import roc_auc_score, precision_recall_curve
    import ast
    import os
    import warnings
    warnings.filterwarnings('ignore')
except ImportError as e:
    print(f"エラー: {e}")
    print("\n必要なパッケージをインストールしてください:")
    print("pip install lightgbm pandas numpy scikit-learn matplotlib")

In [32]:
# インラインで評価クラスを定義
class ProperModelEvaluator:
    """時系列を考慮した適切なモデル評価クラス"""
    
    def __init__(self, base_dir: str = '.'):
        self.base_dir = base_dir
        
    def load_yearly_data(self, years):
        """年単位でデータを読み込む"""
        dfs = []
        for year in years:
            # CSVファイルのパスを試す
            file_paths = [
                f'{self.base_dir}/encoded/{year}encoded_data.csv',
                f'{self.base_dir}/encoded/{year}_encoded_data.csv',
                f'{self.base_dir}/encoded/encoded_data_{year}.csv'
            ]
            
            loaded = False
            for file_path in file_paths:
                if os.path.exists(file_path):
                    df = pd.read_csv(file_path)
                    print(f"Loaded {year} data from {file_path}: {len(df)} rows")
                    dfs.append(df)
                    loaded = True
                    break
            
            if not loaded:
                print(f"Warning: No data file found for {year}")
        
        if not dfs:
            # 複数年のデータが1つのファイルにある場合
            combined_paths = [
                f'{self.base_dir}/encoded/encoded_data.csv',
                f'{self.base_dir}/encoded/2022_2023encoded_data.csv'
            ]
            
            for path in combined_paths:
                if os.path.exists(path):
                    print(f"Found combined data at {path}")
                    df = pd.read_csv(path)
                    
                    # 日付から年を抽出してフィルタリング
                    if '日付' in df.columns:
                        df['year'] = pd.to_datetime(df['日付']).dt.year
                        for year in years:
                            year_df = df[df['year'] == year].copy()
                            if len(year_df) > 0:
                                print(f"Extracted {year} data: {len(year_df)} rows")
                                dfs.append(year_df)
                    break
            
        if not dfs:
            raise ValueError("No data files found")
            
        return pd.concat(dfs, ignore_index=True)
    
    def prepare_features(self, df):
        """特徴量の準備"""
        # 着順を二値分類用に変換（3着以内を1）
        df['target'] = df['着順'].map(lambda x: 1 if x < 4 else 0)
        
        # 不要なカラムを除外
        exclude_cols = ['着順', 'target', 'オッズ', '人気', '上がり', '走破時間', '通過順', 'year']
        feature_cols = [col for col in df.columns if col not in exclude_cols]
        
        return df, feature_cols
    
    def train_model(self, X_train, y_train, X_valid, y_valid):
        """モデルの学習"""
        # クラス比率の計算
        ratio = (y_train == 0).sum() / (y_train == 1).sum()
        
        print(f"\nTraining set:")
        print(f"Negative/Positive ratio: {ratio:.2f}")
        print(f"負例数（3着外）: {(y_train == 0).sum()}")
        print(f"正例数（3着内）: {(y_train == 1).sum()}")
        
        # パラメータ設定
        params = {
            'objective': 'binary',
            'metric': 'binary_logloss',
            'verbosity': -1,
            'boosting_type': 'gbdt',
            'scale_pos_weight': ratio,
            'random_state': 42,
            'num_leaves': 31,
            'learning_rate': 0.05,
            'feature_fraction': 0.9,
            'bagging_fraction': 0.8,
            'bagging_freq': 5,
            'min_child_samples': 20,
            'n_estimators': 300
        }
        
        # モデル学習
        model = lgb.LGBMClassifier(**params)
        model.fit(X_train, y_train)
        
        # 検証データでの予測
        y_pred = model.predict_proba(X_valid)[:, 1]
        auc_score = roc_auc_score(y_valid, y_pred)
        
        # 最適閾値の探索
        precisions, recalls, thresholds = precision_recall_curve(y_valid, y_pred)
        fbeta_scores = (1 + 0.5**2) * (precisions * recalls) / (0.5**2 * precisions + recalls)
        best_idx = np.argmax(fbeta_scores[:-1])
        optimal_threshold = thresholds[best_idx]
        
        print(f"\nValidation AUC: {auc_score:.4f}")
        print(f"Optimal threshold: {optimal_threshold:.4f}")
        
        return model, optimal_threshold

### 1. 単純な時系列分割での評価

まず、既存のデータを正しく時系列で分割して評価します。

In [ ]:
# モデルの学習（時系列分割版）
if 'X_train' in locals():
    evaluator = ProperModelEvaluator()
    model, threshold = evaluator.train_model(X_train, y_train, X_valid, y_valid)
    
    # テストデータでの評価
    y_pred_test = model.predict_proba(X_test)[:, 1]
    test_auc = roc_auc_score(y_test, y_pred_test)
    
    # 混同行列
    TP = ((y_test == 1) & (y_pred_test >= threshold)).sum()
    FP = ((y_test == 0) & (y_pred_test >= threshold)).sum()
    TN = ((y_test == 0) & (y_pred_test < threshold)).sum()
    FN = ((y_test == 1) & (y_pred_test < threshold)).sum()
    
    total = len(y_test)
    
    print(f"\n=== テストセットの結果（時系列分割） ===")
    print(f"テストAUC: {test_auc:.4f}")
    print(f"総レース数: {total}件")
    print(f"正解（3着内を的中）:  {TP}件 ({TP/total*100:.2f}%)")
    print(f"誤検出（3着外を3着内と予測）: {FP}件 ({FP/total*100:.2f}%)")
    print(f"正解（3着外を的中）:  {TN}件 ({TN/total*100:.2f}%)")
    print(f"見逃し（3着内を3着外と予測）: {FN}件 ({FN/total*100:.2f}%)")
    
    # 予測精度の計算
    if TP + FP > 0:
        precision = TP / (TP + FP)
        print(f"\n予測精度（Precision）: {precision:.2%}")
        print(f"→ 3着内と予測した馬のうち、実際に3着内だった割合")
    
    if TP + FN > 0:
        recall = TP / (TP + FN)
        print(f"再現率（Recall）: {recall:.2%}")
        print(f"→ 実際の3着内の馬のうち、予測できた割合")

In [ ]:
# 回収率計算関数
def calculate_returns_time_series(test_data, predictions, threshold):
    """時系列分割したテストデータでの回収率計算"""
    
    # テストデータの年を特定
    test_years = pd.to_datetime(test_data['日付']).dt.year.unique()
    print(f"テストデータの年: {sorted(test_years)}")
    
    # 払戻データを読み込み
    payback_dfs = []
    for year in test_years:
        payback_path = f'payback/{year}.csv'
        if os.path.exists(payback_path):
            df = pd.read_csv(payback_path, encoding='SHIFT-JIS', dtype={'race_id': str})
            df['race_id'] = df['race_id'].str.replace(r'\.0$', '', regex=True)
            payback_dfs.append(df)
            print(f"{year}年の払戻データを読み込みました")
        else:
            print(f"警告: {year}年の払戻データが見つかりません")
    
    if not payback_dfs:
        print("払戻データが見つかりません")
        return None
    
    payback_df = pd.concat(payback_dfs, ignore_index=True)
    payback_df.set_index('race_id', inplace=True)
    
    # 払戻データの変換
    for col in ['単勝', '複勝']:
        if col in payback_df.columns:
            payback_df[col] = payback_df[col].apply(
                lambda x: ast.literal_eval(x) if pd.notna(x) and str(x).strip().startswith('[') else []
            )
    
    # 賭ける馬の決定
    betting_horses = []
    for i in range(len(predictions)):
        if predictions[i] >= threshold:
            race_id = str(int(float(test_data.iloc[i]['race_id'])))
            horse_num = str(int(float(test_data.iloc[i]['馬番'])))
            betting_horses.append((race_id, horse_num))
    
    print(f"\n賭ける馬数: {len(betting_horses)}頭")
    
    # 回収金額の計算
    win_return = 0
    place_return = 0
    
    for race_id, horse_num in betting_horses:
        if race_id in payback_df.index:
            race_data = payback_df.loc[race_id]
            
            # 単勝
            if '単勝' in race_data and isinstance(race_data['単勝'], list):
                win_data = race_data['単勝']
                for j in range(0, len(win_data), 2):
                    if j+1 < len(win_data) and win_data[j] == horse_num:
                        win_return += int(win_data[j + 1].replace(',', ''))
            
            # 複勝
            if '複勝' in race_data and isinstance(race_data['複勝'], list):
                place_data = race_data['複勝']
                for j in range(0, len(place_data), 2):
                    if j+1 < len(place_data) and place_data[j] == horse_num:
                        place_return += int(place_data[j + 1].replace(',', ''))
    
    # 回収率計算
    bet_amount = len(betting_horses) * 100
    win_return_rate = (win_return / bet_amount * 100) if bet_amount > 0 else 0
    place_return_rate = (place_return / bet_amount * 100) if bet_amount > 0 else 0
    
    print(f"\n=== 回収率結果（時系列分割） ===")
    print(f"賭けた回数: {len(betting_horses)}回")
    print(f"投資金額: {bet_amount:,}円")
    print(f"単勝払戻: {win_return:,}円")
    print(f"複勝払戻: {place_return:,}円")
    print(f"単勝回収率: {win_return_rate:.2f}%")
    print(f"複勝回収率: {place_return_rate:.2f}%")
    
    return {
        'win_return_rate': win_return_rate,
        'place_return_rate': place_return_rate,
        'bet_count': len(betting_horses)
    }

# 回収率の計算
if 'y_pred_test' in locals() and 'test_data' in locals():
    returns = calculate_returns_time_series(test_data, y_pred_test, threshold)

# オリジナルの方法との比較
print("\n=== オリジナル手法との比較 ===")
print("\nオリジナル（データリークあり）:")
print("- 学習/テスト分割: ランダムに70/30分割")
print("- 学習データに未来のデータが混入")
print("- 報告された単勝回収率: 130.84%")
print("- 報告された複勝回収率: 132.84%")
print("\n時系列を考慮した正しい分割:")
print("- 学習データ: 過去の日付のみ")
print("- テストデータ: 未来の日付のみ")
print("- データリークなし")
if 'returns' in locals() and returns:
    print(f"- 実際の単勝回収率: {returns['win_return_rate']:.2f}%")
    print(f"- 実際の複勝回収率: {returns['place_return_rate']:.2f}%")
    print("\n【結論】データリークの影響で回収率が大幅に過大評価されていました！")
    
    # 回収率の評価
    print("\n=== 回収率の評価 ===")
    if returns['win_return_rate'] >= 100:
        print("✅ 単勝: 黒字（100%以上）")
    else:
        print(f"❌ 単勝: 赤字（{100 - returns['win_return_rate']:.1f}%の損失）")
    
    if returns['place_return_rate'] >= 100:
        print("✅ 複勝: 黒字（100%以上）")
    else:
        print(f"❌ 複勝: 赤字（{100 - returns['place_return_rate']:.1f}%の損失）")
        
    print("\n【現実的な競馬予測の難しさ】")
    print("- 競馬の控除率（約20-30%）を考慮すると、長期的に100%超えは非常に困難")
    print("- 70-90%の回収率でも、予測モデルとしては優秀")
    print("- データリークなしでの正確な評価が重要")

In [ ]:
# 特徴量重要度を表示
if 'model' in locals():
    importance = model.feature_importances_
    feature_names = X_train.columns
    indices = np.argsort(importance)[::-1]
    
    print("\n=== 重要な特徴量トップ20 ===")
    for i in range(min(20, len(indices))):
        idx = indices[i]
        print(f"{i+1:2d}) {feature_names[idx]:<30} 重要度: {importance[idx]:>6}")

### 3. オリジナルの方法との比較

In [28]:
# オリジナルの方法（データリークあり）での結果を表示
print("\n=== Comparison with Original Method ===")
print("\nOriginal (with data leakage):")
print("- Train/Test split: Random 70/30 split")
print("- Uses future data in training")
print("- Reported win return: 130.84%")
print("- Reported place return: 132.84%")
print("\nProper Time Series Split:")
print("- Train: Earlier dates only")
print("- Test: Later dates only")
print("- No future data leakage")
if 'returns' in locals() and returns:
    print(f"- Actual win return: {returns['win_return_rate']:.2f}%")
    print(f"- Actual place return: {returns['place_return_rate']:.2f}%")
    print("\nThe difference shows the impact of data leakage!")


=== Comparison with Original Method ===

Original (with data leakage):
- Train/Test split: Random 70/30 split
- Uses future data in training
- Reported win return: 130.84%
- Reported place return: 132.84%

Proper Time Series Split:
- Train: Earlier dates only
- Test: Later dates only
- No future data leakage


### 4. 特徴量重要度の確認

In [30]:
# 特徴量重要度を表示
if 'model' in locals():
    importance = model.feature_importances_
    feature_names = X_train.columns
    indices = np.argsort(importance)[::-1]
    
    print("\nTop 20 Important Features:")
    for i in range(min(20, len(indices))):
        idx = indices[i]
        print(f"{i+1:2d}) {feature_names[idx]:<30} {importance[idx]:>6}")